In [8]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#!pip install pandas-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 40.4 MB/s eta 0:00:0000:010:01


In [14]:
#!pip install pyarrow

In [10]:
df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')

In [11]:
df_news_final_project.shape

(200332, 5)

In [12]:
df_news = df_news_final_project

In [13]:
df_news.head()

,url,date,language,title,text
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...
2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce..."
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...
4,http://www.itbusinessnet.com/2020/10/olympus-t...,2020-10-20,en,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...


In [15]:
news = df_news
#news = news.sample(10000,random_state=42)
news.shape

(200332, 5)

In [16]:
#news = news.sample(500,random_state=42)

In [17]:
%%time
import pandas as pd
import re
import string
#!pip install nltk
#!pip install spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 1.66 ms


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
%%time

import re
import nltk
#!pip install pandarallel
from pandarallel import pandarallel

# Assuming you have already installed the pandarallel library

# Initialize pandarallel
pandarallel.initialize()

# Convert 'text' column to string
news['clean_text'] = news['text'].astype(str)

# Remove sentences with more than 80 words
news['clean_text'] = news['clean_text'].parallel_apply(
    lambda x: ' '.join([sentence for sentence in nltk.sent_tokenize(x) if len(sentence.split()) <= 80])
)

# Remove words with more than 20 characters
news['clean_text'] = news['clean_text'].parallel_apply(
    lambda x: ' '.join([word for word in x.split() if len(word) <= 20])
)

# Remove digits
news['clean_text'] = news['clean_text'].apply(lambda x: re.sub(r'[0-9]', '', x))

# Remove uppercase letters in parentheses
news['clean_text'] = news['clean_text'].apply(lambda x: re.sub(r'\([A-Z]\)', '', x))

# Remove non-word and non-space characters
news['clean_text'] = news['clean_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Remove newline characters
news['clean_text'] = news['clean_text'].str.replace('\n', '')

# Optionally convert text to lowercase
# news['clean_text'] = news['clean_text'].str.lower()


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 45.6 s, sys: 14.5 s, total: 1min
Wall time: 1min 16s


In [19]:
%%time

pandarallel.initialize()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
CPU times: user 1.1 ms, sys: 127 µs, total: 1.23 ms
Wall time: 918 µs


In [20]:
pd.set_option('display.max_colwidth', 1000)
news['clean_text'].head(1)

0    Some urban areas of the city started to use ETC system for roadside parking spaces since July   Peoples Daily OnlineLi Wenming Thanks to the application of an artificial intelligence AIempowered roadside electronic toll collection ETC system Chinas capital city Beijing has seen significant improvement in the efficiency of parking fee collection turnover of roadside parking spots order in roadside parking as well as traffic congestion As the city further deepens its roadside parking reform the ETC system has almost covered all the roadside parking spaces in the city with the proportion of vehicles parked on roads using the system exceeding  percent With the AIempowered system drivers can park their vehicles at the parking spots on the roadside and then pay the parking charge via their mobile phones after they drive away This road used to be full of cars and even the normal lanes were occupied You could hardly move a bit during the morning and evening commute time recalled a citizen

In [21]:
keywords = [
  'data science',
  'machine learning',
  'artificial intelligence',
  'natural language processing',
  'computer vision',
  'big data',
  'data mining',
  'deep learning',
  'reinforcement learning',
  'computer vision',
  'natural language generation',
  'text analytics',
  'image recognition',
  'speech recognition',
  'recommendation systems',
  'fraud detection'
]

In [22]:
%%time
import re

filtered_news = news[news['clean_text'].str.contains('|'.join(keywords), flags=re.IGNORECASE)]

CPU times: user 1min 30s, sys: 124 ms, total: 1min 30s
Wall time: 1min 30s


In [23]:
filtered_news.shape

(134051, 6)

In [24]:
#filtered_news.head(1)

In [25]:
import pandas as pd
import pickle

# Write the filtered_news dataframe to a pickle file
filtered_news.to_pickle('keyword_based_filtered_articles_using_re.pkl')

In [26]:
df = filtered_news

### NER Analysis

In [1]:
import pickle

# Open the pickle file in binary mode
with open('keyword_based_filtered_articles_using_re.pkl', 'rb') as file:
    # Load the object from the pickle file
    df = pickle.load(file)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134039 entries, 0 to 200329
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   url         134039 non-null  object
 1   date        134039 non-null  object
 2   language    134039 non-null  object
 3   title       134039 non-null  object
 4   text        134039 non-null  object
 5   clean_text  134039 non-null  object
dtypes: object(6)
memory usage: 7.2+ MB


In [3]:
#!pip install spacy

In [4]:
#!python -m spacy download en_core_web_sm

In [ ]:
%%time

import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

entities = []
labels = []
position_start = []
position_end = []

# Iterate over each text in the 'clean_text' column
for text in df['clean_text']:
    # Apply spaCy NLP pipeline to extract entities
    doc = nlp(text)
    
    # Extract entity details and append to respective lists
    for ent in doc.ents:
        entities.append(ent.text)
        labels.append(ent.label_)
        position_start.append(ent.start_char)
        position_end.append(ent.end_char)
        
# Create a new DataFrame from the extracted entity details
entity_df = pd.DataFrame({'Entities': entities, 'Labels': labels, 'Position_Start': position_start, 'Position_End': position_end})

In [6]:
#!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_md

In [ ]:
# Print the first 20 rows of the entity DataFrame
entity_df.head(20)

In [ ]:
# Extract top organizations
top_organizations = entity_df[entity_df['Labels'] == 'ORG']['Entities'].value_counts().head(10)

# Extract top locations
top_locations = entity_df[entity_df['Labels'] == 'GPE']['Entities'].value_counts().head(10)

In [ ]:
# Print the top organizations
print("Top Organizations:")
print(top_organizations)

In [ ]:
# Print the top locations
print("Top Locations:")
print(top_locations)

In [ ]:
%%time

import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_lg')

entities = []
labels = []
position_start = []
position_end = []

# Iterate over each text in the 'clean_text' column
for text in df['clean_text']:
    # Apply spaCy NLP pipeline to extract entities
    doc = nlp(text)
    
    # Extract entity details and append to respective lists
    for ent in doc.ents:
        entities.append(ent.text)
        labels.append(ent.label_)
        position_start.append(ent.start_char)
        position_end.append(ent.end_char)
        
# Create a new DataFrame from the extracted entity details
entity_df = pd.DataFrame({'Entities': entities, 'Labels': labels, 'Position_Start': position_start, 'Position_End': position_end})

In [37]:
# Extract top organizations
top_organizations = entity_df[entity_df['Labels'] == 'ORG']['Entities'].value_counts().head(10)

# Extract top locations
top_locations = entity_df[entity_df['Labels'] == 'GPE']['Entities'].value_counts().head(10)

In [38]:
# Print the top organizations
print("Top Organizations:")
print(top_organizations)

Top Organizations:
Entities
AI                         372
ChatGPT                    197
Microsoft                  157
Google                     127
Gray Media Group           105
Gray Television Inc         64
GPT                         42
OpenAI                      40
Facebook                    38
Artificial Intelligence     38
Name: count, dtype: int64


In [39]:
# Print the top locations
print("Top Locations:")
print(top_locations)

Top Locations:
Entities
US            220
China          79
PRNewswire     79
India          55
Japan          35
UK             30
Russia         24
New York       23
Canada         21
France         21
Name: count, dtype: int64
